## Decision Tree 
Supervised machine learning algorithm, that resembles flowchart structure, representing decisions and their potential consequences, used for classification and regression

In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 

## problem Statemen 

- We have 3 features for cats and dogs and want to classify them according to these features

|                                                     |   Ear Shape | Face Shape | Whiskers |   Cat  |
|:---------------------------------------------------:|:---------:|:-----------:|:---------:|:------:|
| <img src="images/0.png" alt="drawing" width="50"/> |   Pointy   |   Round     |  Present  |    1   |
| <img src="images/1.png" alt="drawing" width="50"/> |   Floppy   |  Not Round  |  Present  |    1   |
| <img src="images/2.png" alt="drawing" width="50"/> |   Floppy   |  Round      |  Absent   |    0   |
| <img src="images/3.png" alt="drawing" width="50"/> |   Pointy   |  Not Round  |  Present  |    0   |
| <img src="images/4.png" alt="drawing" width="50"/> |   Pointy   |   Round     |  Present  |    1   |
| <img src="images/5.png" alt="drawing" width="50"/> |   Pointy   |   Round     |  Absent   |    1   |
| <img src="images/6.png" alt="drawing" width="50"/> |   Floppy   |  Not Round  |  Absent   |    0   |
| <img src="images/7.png" alt="drawing" width="50"/> |   Pointy   |  Round      |  Absent   |    1   |
| <img src="images/8.png" alt="drawing" width="50"/> |    Floppy  |   Round     |  Absent   |    0   |
| <img src="images/9.png" alt="drawing" width="50"/> |   Floppy   |  Round      |  Absent   |    0   |


Encoding the categorical features:

- Ear Shape: Pointy = 1, Floppy = 0
- Face Shape: Round = 1, Not Round = 0
- Whiskers: Present = 1, Absent = 0

## Data set

In [2]:
X_train = np.array([[1, 1, 1],
[0, 0, 1],
 [0, 1, 0],
 [1, 0, 1],
 [1, 1, 1],
 [1, 1, 0],
 [0, 0, 0],
 [1, 1, 0],
 [0, 1, 0],
 [0, 1, 0]])

y_train = np.array([1, 1, 0, 0, 1, 1, 0, 1, 0, 0])

In [3]:
y_train[X_train[0]]

array([1, 1, 1])

## Build DT functions from scratch 

### 1- Entropy <br>

$$H(p_1) = -p_1 \text{log}_2(p_1) - (1- p_1) \text{log}_2(1- p_1)$$

#### The higher the entropy, the lower the purity

In [4]:
y_train[y_train==1]

array([1, 1, 1, 1, 1])

In [5]:
def compute_entropy(y):
    entropy = 0
    if len(y) != 0:
        p = sum(y) / len(y)
        if p == 1 or p == 0:
            entropy = 0
        elif p == .5:
            entropy = 1
        else:
            entropy = -p * np.log2(p) - (1-p) * np.log2(1-p)
    else:
         entropy = 0
    
    return entropy


In [6]:
# Test function
a = [1,1,1,0, 0]
compute_entropy(a)

0.9709505944546686

### 2- ٍsplit indices

Given a dataset and a index feature, return two lists for the two split nodes <br> the left node has the animals that have 
that feature = 1 and the right node those that have the feature = 0 <br>
index feature = 0 => ear shape <br>
index feature = 1 => face shape <br>
index feature = 2 => whiskers <br>


In [7]:
def split_indices(X, node_indices,feature):
    left_indices = []
    right_indices = []
    
    for i in node_indices:
        if X[i, feature] == 1:
            left_indices.append(i)
        else:
            right_indices.append(i)
            
    return left_indices, right_indices  

In [8]:
# Test function
root = np.arange(10)
l,r = split_indices(X_train,root, 0) # ear shape feature as a root node 

print(f"left indices are => {l}")
print(f"right indices are => {r}")

left indices are => [0, 3, 4, 5, 7]
right indices are => [1, 2, 6, 8, 9]


In [9]:
# Now split in lesft incisces
ll,rl = split_indices(X_train,l, 1) # face shape
print(f"left indices are => {ll}")
print(f"right indices are => {rl}")

left indices are => [0, 4, 5, 7]
right indices are => [3]


In [10]:
y_train[ll] #then if pointy and round then its a cat

array([1, 1, 1, 1])

### 3- Information Gain

$$\text{Information Gain} = H(p_1^\text{node})- (w^{\text{left}}H(p_1^\text{left}) + w^{\text{right}}H(p_1^\text{right}))$$


In [11]:
def information_gain(X, y, node_indices, feature):
    information_gain = 0
    
    left_indices, right_indices = split_indices(X, node_indices,feature)
    
    w_left  = len(left_indices) / len(X) # w indicate to number of elements in node / number of elements in root node
    w_right = len(right_indices) / len(X)
    
    if len(left_indices) == 0 or len(right_indices) == 0:
        p_left = p_right = 0
    else:
        p_left  = sum(y[left_indices]) / len(left_indices)
        p_right = sum(y[right_indices]) / len(right_indices)
    
    root_entropy  = compute_entropy(y)
    left_entropy  = compute_entropy(y[left_indices])
    right_entropy = compute_entropy(y[right_indices])
    
    information_gain = root_entropy - (w_left * left_entropy + w_right * right_entropy)
    
    return information_gain

In [12]:
# Test function 
root = np.arange(10)
info_gain_ear_shape  = information_gain(X_train, y_train, root, 0 )
info_gain_face_shape = information_gain(X_train, y_train, root, 1 )
info_gain_whiskers   = information_gain(X_train, y_train, root, 2 )

print(f"information_gain for ear shape  = {info_gain_ear_shape:.2f}")
print(f"information_gain for face shape = {info_gain_face_shape:.2f}")
print(f"information_gain for whiskers   = {info_gain_whiskers:.2f}")

information_gain for ear shape  = 0.28
information_gain for face shape = 0.03
information_gain for whiskers   = 0.12


### 4- Get best split

In [13]:
X_train.shape[1]

3

In [14]:
def get_best_split(X, y, node_indices):
    num_of_features = X.shape[1]

    best_feature = 0
    max_info_gain = 0
    
    for i in range(num_of_features):
        info_gain = information_gain(X, y, node_indices, i)
        
        if max_info_gain < info_gain:
            max_info_gain = info_gain
            best_feature = i
    return best_feature

In [15]:
# Test function 
print(f"best feature is {get_best_split(X_train, y_train, root)}")

best feature is 0


In [16]:
# in
print(f"best feature in side of ear shape is pointy {get_best_split(X_train, y_train, l)}")

best feature in side of ear shape is pointy 1


## Building the tree

In [17]:
# Not graded
tree = []

def build_tree_recursive(X, y, node_indices, branch_name, max_depth, current_depth):
    """
    Build a tree using the recursive algorithm that split the dataset into 2 subgroups at each node.
    This function just prints the tree.
    
    Args:
        X (ndarray):            Data matrix of shape(n_samples, n_features)
        y (array like):         list or ndarray with n_samples containing the target variable
        node_indices (ndarray): List containing the active indices. I.e, the samples being considered in this step.
        branch_name (string):   Name of the branch. ['Root', 'Left', 'Right']
        max_depth (int):        Max depth of the resulting tree. 
        current_depth (int):    Current depth. Parameter used during recursive call.
   
    """ 

    # Maximum depth reached - stop splitting
    if current_depth == max_depth:
        formatting = " "*current_depth + "-"*current_depth
        print(formatting, "%s leaf node with indices" % branch_name, node_indices)
        return
   
    # Otherwise, get best split and split the data
    # Get the best feature and threshold at this node
    best_feature = get_best_split(X, y, node_indices) 
    
    formatting = "-"*current_depth
    print("%s Depth %d, %s: Split on feature: %d" % (formatting, current_depth, branch_name, best_feature))
    
    # Split the dataset at the best feature
    left_indices, right_indices = split_indices(X, node_indices, best_feature)
    tree.append((left_indices, right_indices, best_feature))
    
    # continue splitting the left and the right child. Increment current depth
    build_tree_recursive(X, y, left_indices, "Left", max_depth, current_depth+1)
    build_tree_recursive(X, y, right_indices, "Right", max_depth, current_depth+1)

In [18]:
build_tree_recursive(X_train, y_train, root, "Root", max_depth=2, current_depth=0)


 Depth 0, Root: Split on feature: 0
- Depth 1, Left: Split on feature: 1
  -- Left leaf node with indices [0, 4, 5, 7]
  -- Right leaf node with indices [3]
- Depth 1, Right: Split on feature: 2
  -- Left leaf node with indices [1]
  -- Right leaf node with indices [2, 6, 8, 9]
